# VONA Spike Test

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Button, Layout

import csv
import datetime as dt

from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4, landscape

import textwrap

## Fixed values

### Item No

In [ ]:
itemno = ['Item', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16']

### Element

In [ ]:
element = ['Message title:', 'Issued:', 'Volcano:', 'Current Aviation Color Code:', 
           'Previous Aviation Color Code:', 'Source:', 'Notice Number:', 'Volcano Location:', 
           'Area:', 'Summit Elevation:', 'Volcanic Activity Summary:', 'Volcanic Cloud Height:', 
           'Other Volcanic Cloud Information:', 'Remarks:', 'Contacts:', 'Next Notice:']

### Content

In [ ]:
noticetitle = 'VOLCANO OBSERVATORY NOTICE FOR AVIATION (VONA)'
source = 'GNS Science, New Zealand'
contacts = 'Volcano Duty Officer, +6473748211ph,+6473748199fax'
nextnotice = 'Will be issued if there is a change in aviation color code or when a significant volcanic event happens within the current color code'


content = ['VOLCANO OBSERVATORY NOTICE FOR AVIATION', 'date-time', 'volcanoname', 'col1', 'col2', 
           'GNS Science, New Zealand', '', 'location', 'area', '', '', '', '', '',
           'Volcano Duty Officer, +6473748211ph,+6473748199fax',
           'Will be issued if there is a change in aviation color code or when a significant volcanic event happens within the current color code'
          ]

In [ ]:
# data= [['Item No', 'Element', 'Content'],
#     ['1', 'Message title:', 'VOLCANO OBSERVATORY NOTICE FOR AVIATION'],
#     ['2', 'Issued:', ''],
#     ['3', 'Volcano:', ''],
#     ['4', 'Current Aviation Color Code:', ''],
#     ['5', 'Previous Aviation Color Code:', ''],
       
#               ]

## Initial values for widget functions, if not adjusted

In [ ]:
#vonanumber
vonanum = 999

#vonanumber notice detail
vonanotice = 'NZ VONA'

## Functions for widget actions

In [ ]:
wid = 80

def selectvolcano(s):
    global volcano
    volcano = s['new']
    
def selectcurrentcol(c):
    global curcol
    curcol = c['new'].upper()
    
def selectpreviouscol(p):
    global precol
    precol = p['new']
    
def vonanumber(n):
    global vonanum
    vonanum = n['new']
    
def summarytext(summ):
    global summtxt
    txt = summ['new']
    wrapper = textwrap.TextWrapper(width=wid)
    summtxt = wrapper.wrap(text=txt)
    
#volcanic cloud height
def cloudheight(clht):
    global cloudht
    txt = clht['new']
    wrapper = textwrap.TextWrapper(width=wid)
    cloudht = wrapper.wrap(text=txt)
    
#other volcanic cloud information
def cloudinfo(clin):
    global cloudin
    txt = clin['new']
    wrapper = textwrap.TextWrapper(width=wid)
    cloudin = wrapper.wrap(text=txt)

#remarks
def remarks(rem):
    global remark
    txt = rem['new'] 
    wrapper = textwrap.TextWrapper(width=wid)
    remark = wrapper.wrap(text=txt)

def process(z):
    global volcdata
    f = open('vona_data.csv', 'r')
    reader = csv.reader(f)
    volcdata = {}
    for row in reader:
        volcdata[row[0]] = {'name_id':row[1], 'location':row[2], 'area':row[3], 'height':row[4]}
    (issuetime, volcano_name, noticenum, location, area, summit) = genoutput(volcano)
#     print ('inside process', issuetime, volcano_name, noticenum, location, area, summit)
#     outputtxt()
#     outputpdf()
    outputpdf4(issuetime, volcano_name, noticenum, location, area, summit)

## Other functions

In [ ]:
def genoutput(volcano):
    utcnow = dt.datetime.utcnow()
    #issue time
    issuetime = utcnow.strftime('%Y%m%d/%H%MZ')
    #issue year, for notice number
    issueyear = utcnow.strftime('%Y')
    #volcano name and id code
    volcano_name = volcdata[volcano]['name_id']
    #vona number
    num = str(vonanum).zfill(2)
    noticenum = vonanotice+' ' +issueyear+'/'+num
    #volcano location
    location = volcdata[volcano]['location']
    #area
    area = volcdata[volcano]['area']
    #summit elevation
    summit = volcdata[volcano]['height']
#     content[10] = summtxt
#     content[11] = cloudht
#     content[12] = cloudinfo
#     content[13] = remark
    return (issuetime, volcano_name, noticenum, location, area, summit)

def outputtxt():
    f = open('vona.txt', 'w')
#     print (noticetitle,'\n')
    f.write(noticetitle)
    f.write('\n')
    #print each line
    for idx,(e,c) in enumerate(zip(element, content)):
#         print (f"{idx:<5}{e:<40}{c:<100}")
        f.write (f"{idx:<5}{e:<40}{c:<100}\n")
    f.close()
    
def outputpdf():
    #text only
    left_margin = 50
    first_row = 50
    row_increment = 20
    row = first_row + row_increment
    canv = canvas.Canvas('vona.pdf', bottomup=0, initialFontName='Courier', initialFontSize=10)
    canv.drawString(left_margin,first_row,noticetitle)
    for idx,(e,c) in enumerate(zip(element, content)):
        txtstring = f"{idx:<5}{e:<40}{c:<100}\n"
        canv.drawString(left_margin, row, txtstring)
        row += row_increment
    canv.save()
    
def outputpdf2():
    #text and grid, lines drawn individually
    left_margin = 50
    first_row = 50
    row_increment = 20
    page_width = 800
    page_bottom = 400
    canv = canvas.Canvas('vona2.pdf', bottomup=0)
    canv.setFont('Courier', 10)
    canv.setPageSize(landscape(A4))
    canv.setLineWidth(.3)
    #draw outer box
    canv.line(left_margin-1, first_row+10, page_width, first_row+10)
    canv.line(page_width, first_row+10, page_width, page_bottom)
    canv.line(page_width, page_bottom, left_margin-1, page_bottom)
    canv.line(left_margin-1, page_bottom, left_margin-1, first_row+10)
    #vertical lines
    canv.line(left_margin+20, first_row+10, left_margin+20, page_bottom)
    canv.line(left_margin+250, first_row+10, left_margin+250, page_bottom)
    
    #add text
    row = first_row + row_increment
    canv.drawString(left_margin,first_row,noticetitle)
    for idx,(e,c) in enumerate(zip(element, content)):
        txtstring = f"{idx:<5}{e:<40}{c:<100}\n"
        canv.drawString(left_margin, row, txtstring)
        #draw line below each text
        canv.line(left_margin-1, row+2, page_width, row+2)
        row += row_increment
    canv.save()
    

    
def outputpdf4(issuetime, volcano_name, noticenum, location, area, summit):
    issuetime = issuetime
    volcano_name = volcano_name
    noticenum = noticenum
    location=location
    area = area
    summit = summit
    
    #column positions, row details
    page_width = 800
    page_bottom = 400
    col1 = 50
    col2 = col1 + 30
    col3 = col1 + 250
    page_width = 800
    first_row = 50
    row_inc = 20
    line_inc = 5
    row = first_row
    
    #canvas
    canv = canvas.Canvas('vona4.pdf', bottomup=0)
    canv.setFont('Courier', 10)
    canv.setPageSize(landscape(A4))
    canv.setLineWidth(.3)
           
    #header
    canv.drawString(col1,row,noticetitle)
    
    #message title
    row += row_inc
    canv.drawString(col1,row,itemno[0])
    canv.drawString(col2,row,element[0])
    canv.drawString(col3,row,content[0])
    canv.line(col1-1, row+line_inc, page_width, row+line_inc)
    
    #issue time
    row += row_inc
    canv.drawString(col1,row,itemno[1])
    canv.drawString(col2,row,element[1])
    canv.drawString(col3,row,issuetime)
    canv.line(col1-1, row+line_inc, page_width, row+line_inc)
    
    #volcano name
    row += row_inc
    canv.drawString(col1,row,itemno[2])
    canv.drawString(col2,row,element[2])
    canv.drawString(col3,row,volcano_name)
    canv.line(col1-1, row+line_inc, page_width, row+line_inc)
    
    #current color code
    row += row_inc
    canv.drawString(col1,row,itemno[3])
    canv.drawString(col2,row,element[3])
    canv.drawString(col3,row,curcol)
    canv.line(col1-1, row+line_inc, page_width, row+line_inc)
    
    #previous color code
    row += row_inc
    canv.drawString(col1,row,itemno[4])
    canv.drawString(col2,row,element[4])
    canv.drawString(col3,row,precol)
    canv.line(col1-1, row+line_inc, page_width, row+line_inc)
    
    #source color code
    row += row_inc
    canv.drawString(col1,row,itemno[5])
    canv.drawString(col2,row,element[5])
    canv.drawString(col3,row,source)
    canv.line(col1-1, row+line_inc, page_width, row+line_inc)
    
    #notice number
    row += row_inc
    canv.drawString(col1,row,itemno[6])
    canv.drawString(col2,row,element[6])
    canv.drawString(col3,row,noticenum)
    canv.line(col1-1, row+line_inc, page_width, row+line_inc)
    
    #location
    row += row_inc
    canv.drawString(col1,row,itemno[7])
    canv.drawString(col2,row,element[7])
    canv.drawString(col3,row,location)
    canv.line(col1-1, row+line_inc, page_width, row+line_inc)
    
    #area
    row += row_inc
    canv.drawString(col1,row,itemno[8])
    canv.drawString(col2,row,element[8])
    canv.drawString(col3,row,area)
    canv.line(col1-1, row+line_inc, page_width, row+line_inc)
    
    #summit elevation
    row += row_inc
    canv.drawString(col1,row,itemno[9])
    canv.drawString(col2,row,element[9])
    canv.drawString(col3,row,summit)
    canv.line(col1-1, row+line_inc, page_width, row+line_inc)
    
    #summtxt
    row += row_inc
    canv.drawString(col1,row,itemno[10])
    canv.drawString(col2,row,element[10])
    row -= row_inc
    for line in summtxt:
        row += row_inc
        canv.drawString(col3,row,line)
    canv.line(col1-1, row+line_inc, page_width, row+line_inc)
    
    #cloudht
    row += row_inc
    canv.drawString(col1,row,itemno[11])
    canv.drawString(col2,row,element[11])
    row -= row_inc
    for line in cloudht:
        row += row_inc
        canv.drawString(col3,row,line)
    canv.line(col1-1, row+line_inc, page_width, row+line_inc)
    
    #cloudinfo
    row += row_inc
    canv.drawString(col1,row,itemno[12])
    canv.drawString(col2,row,element[12])
    row -= row_inc
    for line in cloudin:
        row += row_inc
        canv.drawString(col3,row,line)
    canv.line(col1-1, row+line_inc, page_width, row+line_inc)
    
    #remark
    row += row_inc
    canv.drawString(col1,row,itemno[13])
    canv.drawString(col2,row,element[13])
    row -= row_inc
    for line in remark:
        row += row_inc
        canv.drawString(col3,row,line)
    canv.line(col1-1, row+line_inc, page_width, row+line_inc)
    
    #contacts
    row += row_inc
    canv.drawString(col1,row,itemno[14])
    canv.drawString(col2,row,element[14])
    canv.drawString(col3,row,contacts)
    canv.line(col1-1, row+line_inc, page_width, row+line_inc)
    
    #nextnotice
    row += row_inc
    canv.drawString(col1,row,itemno[15])
    canv.drawString(col2,row,element[15])
    canv.drawString(col3,row,nextnotice)
    canv.line(col1-1, row+line_inc, page_width, row+line_inc)
    
    #draw outer box
    canv.line(col1-1, first_row+10, page_width, first_row+10)
    canv.line(page_width, first_row+10, page_width, row+line_inc)
    canv.line(col1-1, row+line_inc, col1-1, first_row+10)
#     #vertical lines
#     canv.line(col1+20, first_row+10, col1+20, page_bottom)
#     canv.line(col1+250, first_row+10, col1+250, page_bottom) 
    
    
    canv.save()

## Display widgets

In [ ]:
style = {'description_width': 'initial'}

s = widgets.Select(
    options=['Choose one', 'White Island', 'Ruapehu'],
    value='Choose one',
    description='Volcano'
    )
s.observe(selectvolcano, names='value')

c = widgets.Select(
    options=['Choose one', 'green', 'yellow', 'orange', 'red'],
    value='Choose one',
    description='Current color'
    )
c.observe(selectcurrentcol, names='value')

p = widgets.Select(
    options=['Choose one', 'green', 'yellow', 'orange', 'red'],
    value='Choose one',
    description='Previous color',
    style=style
    )
p.observe(selectpreviouscol, names='value')

n = widgets.IntSlider(
    value=1, min=1, max=20, step=1,
    description='VONA number', style=style,
    disabled=False, continuous_update=False, 
    orientation='horizontal', readout=True,
    slider_color='white')
n.observe(vonanumber, names='value')

#activity summary text box
summ = widgets.Textarea(
    value='',
    placeholder='',
    description='Volcanic Activity Summary',
    style=style,
    disabled=False
)
summ.observe(summarytext, names='value')

#cloud height text box
clht = widgets.Textarea(
    value='',
    placeholder='',
    description='Volcanic cloud height',
    style=style,
    disabled=False
)
clht.observe(cloudheight, names='value')

#other cloud information
clin = widgets.Textarea(
    value='',
#     placeholder='Type text description',
    description='Other cloud info',
    style=style,
    disabled=False
)
clin.observe(cloudinfo, names='value')

#remarks
rem = widgets.Textarea(
    value='',
    placeholder='',
    description='remarks',
    style=style,
    disabled=False
)
rem.observe(remarks, names='value')

#process input button
z = widgets.Button(
    description='Process',
    tooltip='Create VONA')
z.on_click(process)

#arrange widgets in a sensible order
row1 = widgets.HBox([s,n])
row2 = widgets.HBox([c,p])
row3 = widgets.HBox([summ,clht,clin])
row4 = widgets.HBox([rem])
rowlast = widgets.HBox([z])
widgets.VBox([row1,row2,row3,row4,rowlast])